# PMD: Elasto-plasticity with isotropic hardening

Associated, rate-independent plasticity. Standard Lagrangian is sufficient.

Application of GSM MPDP - with zero rate-independent dissipation potential

This notebook is the starting point towards generalization of GSM MDPD for
 - rate-dependent models
 - non-associated models

In [ ]:
%matplotlib widget
from bmcs_matmod.gsm.gsm_mpdp import GSMMPDP
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
from bmcs_utils.api import Cymbol
sp.init_printing()

### Helmholtz free energy

In [ ]:

E = sp.Symbol(r'E', codename='E_', real=True, nonnegative=True)
K = sp.Symbol(r'K', codename='K_', real=True)
f_c = sp.Symbol(r'f_\mathrm{c}', codename='f_c_')

mparams = (E, K, f_c)

# ## External state variables

eps = sp.Symbol(r'\varepsilon', codename='eps_', real=True)
eps_a = sp.Matrix([eps])
sig = sp.Symbol(r'\sigma', codename='sig_', real=True)
sig_a = sp.Matrix([sig])

# ## Internal state variables

eps_p = sp.Symbol(r'\varepsilon^\mathrm{p}', codename='eps_p_', real=True)
eps_p_a = sp.Matrix([eps_p])
sig_p = sp.Symbol(r'\sigma^\mathrm{p}', codename='sig_p_', real=True)
sig_p_a = sp.Matrix([sig_p])

z = sp.Symbol(r'z', codename='z_', real=True, nonnegative=True)
z_a = sp.Matrix([z])
Z = sp.Symbol(r'Z', codename='Z_', real=True, nonnegative=True)
Z_a = sp.Matrix([Z])

# ## Free energy potential
eps_el = eps - eps_p
U_e_ = sp.Rational(1,2) * E * eps_el**2
U_p_ =  sp.Rational(1,2) * K * z**2
F_ = U_e_ + U_p_

# ## Flow potential
f_ = sp.sqrt(sig_p**2) - (f_c + Z)
phi_ext_ = 0

Eps_vars = (eps_p_a, z_a)
Sig_vars = (sig_p_a, Z_a)
Sig_signs =  (-1, 1)


### GSM drivers

In [ ]:
gsm_vep = GSMMPDP(
    name = 'gsm_F_1d_mpdp_ep_lih',
    u_vars = eps_a,
    sig_vars = sig_a,
    m_params = mparams,
    Eps_vars = Eps_vars,
    Sig_vars = Sig_vars,
    Sig_signs = Sig_signs,
    F_expr = F_,
    f_expr = f_,
    phi_ext_expr = phi_ext_,
)

In [ ]:
gsm_vep.Sig_, gsm_vep.phi_.diff(gsm_vep.Sig.as_explicit()), gsm_vep.Y_, gsm_vep.Phi_

In [ ]:
(gamma_mech, L_, dL_dS_, dL_dS_A_, dR_dA_n1), (eps_n, delta_eps, Eps_n, delta_A, delta_t, _, _), Sig_n1, f_n1, R_n1, dR_dA_OI_n1 = gsm_vep.Sig_f_R_dR_n1
gamma_mech

In [ ]:
L_

In [ ]:
dL_dS_

In [ ]:
R_n1_subs = R_n1.replace(sp.sign, lambda x: 1)
R_n1_subs

In [ ]:
delta_A

In [ ]:
dR_dA_n1_subs = dR_dA_n1.replace(sp.sign, lambda x: 1)
dR_dA_n1_subs

In [ ]:
Sig_n1

## Monotonic strain-driven loading

In [ ]:
mparams = {E:1, K:0.1, f_c:1}
margs = [mparams[p] for p in mparams]
n_t = 5
n_I = 1
eps_max = 2
t_t = np.linspace(0, 10, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
Eps_n_Ib = np.zeros((n_I, gsm_vep.n_Eps), dtype=np.float64)
t_t, eps_ta_nr_coarse, Eps_t, Sig_t_nr_coarse, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_vep.get_response(eps_n_t, t_t, 3, *margs)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(eps_ta_nr_coarse, Sig_t_nr_coarse[:,:,0], color='red')

In [ ]:
%matplotlib widget
mparams = {E:1, K:1, f_c:1}
margs = [mparams[p] for p in mparams]
n_t = 200
n_I = 1
eps_max = 3
t_t = np.linspace(0, 1, n_t)

eps_n_t = np.linspace(0, eps_max, n_t)
t_t = np.hstack([t_t, np.linspace(1, 2, n_t)[1:], np.linspace(2, 3, n_t)[1:]])
eps_n_t = np.hstack([eps_n_t, eps_n_t[-1] - 0 * eps_n_t[1:], eps_n_t[-1::-1]])
fig, ax = plt.subplots(1,1)
ax.plot(eps_n_t)

In [ ]:
t_t, eps_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_vep.get_response(eps_n_t, t_t, 100, *margs)
# Plot viscoplastic strain and damage along time
eps_t = eps_ta[:, 0]
eps_p_t, z_t = Eps_t[:, 0, :].T
sig_p_t, Z_t = Sig_t[:, 0, :].T

fig, (ax_sig, ax_omega) = plt.subplots(1, 2, figsize=(12, 4))

# Plot stress-strain and iterations-strain curves
ax_iter = ax_sig.twinx()
ax_eps = ax_omega.twinx()

ax_sig.plot(eps_t, sig_p_t, color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_p_t, Sig_t[:,:,0], ls='dotted', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray', lw=0.5)
ax_iter.set_ylabel(r'$k$', color='gray')

ax_omega.plot(t_t, z_t, label='Damage', color='brown')
ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.legend()
ax_omega.set_xlabel(r'$t$')
ax_omega.set_ylabel(r'$z$', color='brown')
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

plt.tight_layout()
plt.show()

## Vectorized evaluation

In [ ]:
mparams = {E:1, K:0.1, f_c:1}
margs = [mparams[p] for p in mparams]
eps_n_I = np.array([0, 1.0, 1.0, -1.0, 0, 0], dtype=np.float64)
d_eps_I = np.array([0.1, -0.1, 0.1, -0.1, 2, -2], dtype=np.float64)
n_I = eps_n_I.shape[0]
d_t = 1
Eps_n_Ib = np.zeros((n_I, gsm_vep.n_Eps), dtype=np.float64)

Eps_n1_Ib, Sig_n1_Ib, lam_I, k_I = gsm_vep.get_state_n1(eps_n_I, d_eps_I, d_t, Eps_n_Ib, 30, *margs )
Eps_n1_Ib, Sig_n1_Ib, lam_I, k_I